In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor
import optuna
from optuna.samplers import TPESampler

In [ ]:
# Configure problem number
PROBLEM_NUM = 36

X_path = f"./data_31_40/problem_{PROBLEM_NUM}/dataset_{PROBLEM_NUM}.csv"
y_path = f"./data_31_40/problem_{PROBLEM_NUM}/target_{PROBLEM_NUM}.csv"
Xeval_path = f"./data_31_40/problem_{PROBLEM_NUM}/EVAL_{PROBLEM_NUM}.csv"

X = pd.read_csv(X_path)
y = pd.read_csv(y_path)
X_eval = pd.read_csv(Xeval_path)

y1 = y["target01"]

print(f"Problem {PROBLEM_NUM}")
print(f"X: {X.shape}, y1: {y1.shape}, X_eval: {X_eval.shape}")

# Create train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y1, test_size=0.2, random_state=42, shuffle=True
)

print(f"\nTrain/Val Split:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

In [ ]:
# Get top 8 features by training a quick baseline model
print("Training baseline model to extract top 8 features...")
print("="*80)

model_baseline = CatBoostRegressor(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=0
)

model_baseline.fit(X_train, y_train)

# Get feature importances
feature_importance = model_baseline.get_feature_importance()
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Extract top 8 features
top_8_features = importance_df.head(8)['feature'].tolist()

print("\nTop 8 Features Selected:")
print("="*80)
for i, feat in enumerate(top_8_features, 1):
    imp = importance_df[importance_df['feature'] == feat]['importance'].values[0]
    print(f"{i}. {feat:<20} (importance: {imp:.4f})")

# Create subsets with only top 8 features
X_train_top8 = X_train[top_8_features]
X_val_top8 = X_val[top_8_features]
X_eval_top8 = X_eval[top_8_features]

print(f"\nSubset shapes:")
print(f"X_train_top8: {X_train_top8.shape}")
print(f"X_val_top8: {X_val_top8.shape}")
print(f"X_eval_top8: {X_eval_top8.shape}")
print("="*80)

In [ ]:
# Define Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.0, 2.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 10.0),
        'random_state': 42,
        'verbose': 0
    }
    
    # Train model with suggested parameters
    model = CatBoostRegressor(**params)
    
    model.fit(
        X_train_top8, y_train,
        eval_set=(X_val_top8, y_val),
        early_stopping_rounds=50,
        verbose=0
    )
    
    # Evaluate on validation set
    y_val_pred = model.predict(X_val_top8)
    val_r2 = r2_score(y_val, y_val_pred)
    
    return val_r2  # Maximize validation R²

print("Objective function defined.")
print("Ready to run Optuna optimization...")

In [ ]:
# Run Optuna optimization
print("="*80)
print("STARTING OPTUNA OPTIMIZATION (8 FEATURES ONLY)")
print("="*80)
print("Objective: Maximize Validation R²")
print(f"Features: {len(top_8_features)}")
print(f"Trials: 150")
print("="*80)

# Create study
study = optuna.create_study(
    direction='maximize',  # Maximize R²
    sampler=TPESampler(seed=42),
    study_name='catboost_optuna_8feat_p36'
)

# Optimize
study.optimize(objective, n_trials=150, show_progress_bar=True)

print("\n" + "="*80)
print("OPTIMIZATION COMPLETE!")
print("="*80)

In [ ]:
# Display best results
print("\n" + "="*80)
print("BEST TRIAL RESULTS")
print("="*80)
print(f"Best Trial Number: {study.best_trial.number}")
print(f"Best Validation R²: {study.best_value:.6f}")

print("\nBest Hyperparameters:")
print("-"*80)
for param, value in study.best_params.items():
    print(f"{param:<25} {value}")
print("="*80)

In [ ]:
# Train final model with best parameters
print("\n" + "="*80)
print("TRAINING FINAL MODEL WITH BEST PARAMETERS")
print("="*80)

best_params = study.best_params.copy()
best_params['random_state'] = 42
best_params['verbose'] = 100

model_best = CatBoostRegressor(**best_params)

model_best.fit(
    X_train_top8, y_train,
    eval_set=(X_val_top8, y_val),
    early_stopping_rounds=50,
    plot=False
)

print("\n" + "="*80)
print("Training Complete!")
print("="*80)

In [ ]:
# Evaluate final model
y_train_pred = model_best.predict(X_train_top8)
y_val_pred = model_best.predict(X_val_top8)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
train_r2 = r2_score(y_train, y_train_pred)
val_r2 = r2_score(y_val, y_val_pred)

print("\n" + "="*80)
print("FINAL MODEL PERFORMANCE (8 FEATURES, OPTUNA OPTIMIZED)")
print("="*80)
print(f"{'Metric':<20} {'Train':<15} {'Validation':<15} {'Gap':<15}")
print("="*80)
print(f"{'R²':<20} {train_r2:<15.6f} {val_r2:<15.6f} {train_r2 - val_r2:<15.6f}")
print(f"{'RMSE':<20} {train_rmse:<15.6f} {val_rmse:<15.6f} {val_rmse - train_rmse:<15.6f}")
print("="*80)

if val_rmse - train_rmse > 0.05:
    print("\n⚠️ Warning: Model shows signs of overfitting (RMSE gap > 0.05)")
else:
    print("\n✓ Excellent generalization achieved (RMSE gap < 0.05)")

print(f"\n📊 Model Summary:")
print(f"   Features used: {len(top_8_features)} / 273 ({len(top_8_features)/273*100:.1f}%)")
print(f"   Validation R²: {val_r2:.6f}")
print(f"   Validation RMSE: {val_rmse:.6f}")
print(f"   Trials completed: {len(study.trials)}")

In [ ]:
# Visualize optimization history
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Optimization history
trial_numbers = [trial.number for trial in study.trials]
trial_values = [trial.value for trial in study.trials]

axes[0].plot(trial_numbers, trial_values, marker='o', linestyle='-', alpha=0.6, markersize=4)
axes[0].axhline(y=study.best_value, color='red', linestyle='--', linewidth=2, label=f'Best R²: {study.best_value:.6f}')
axes[0].set_xlabel('Trial Number', fontsize=12)
axes[0].set_ylabel('Validation R²', fontsize=12)
axes[0].set_title('Optuna Optimization History', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Parameter importance (if available)
try:
    param_importance = optuna.importance.get_param_importances(study)
    params = list(param_importance.keys())
    importances = list(param_importance.values())
    
    axes[1].barh(params, importances)
    axes[1].set_xlabel('Importance', fontsize=12)
    axes[1].set_title('Hyperparameter Importance', fontsize=14, fontweight='bold')
    axes[1].grid(axis='x', alpha=0.3)
except:
    axes[1].text(0.5, 0.5, 'Parameter importance\nrequires multiple trials', 
                ha='center', va='center', fontsize=12)
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)
    axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete!")

In [ ]:
# Generate predictions on evaluation set
print("\n" + "="*80)
print("Generating predictions on evaluation set...")
print("="*80)

eval_pred = model_best.predict(X_eval_top8)

# Save predictions
output_filename = f"EVAL_target01_{PROBLEM_NUM}_catboost_optuna_8feat.csv"
submission = pd.DataFrame({"target01": eval_pred})
submission.to_csv(output_filename, index=False)

print(f"\n✓ Saved: {output_filename}")
print(f"Predictions shape: {eval_pred.shape}")
print(f"Predictions range: [{eval_pred.min():.6f}, {eval_pred.max():.6f}]")
print(f"\nExpected Test RMSE: ~{val_rmse:.6f} (based on validation)")
print(f"\nFeatures used:")
for i, feat in enumerate(top_8_features, 1):
    print(f"  {i}. {feat}")

In [ ]:
# Display top 10 trials
print("\n" + "="*80)
print("TOP 10 TRIALS")
print("="*80)

trials_df = study.trials_dataframe()
trials_df = trials_df.sort_values('value', ascending=False).head(10)

# Show only trial number and value
print(trials_df[['number', 'value']].to_string(index=False))
print("="*80)